<a href="https://colab.research.google.com/github/chihchao/AoP/blob/main/crawler_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 高雄紅橘線捷運車站中心座標

dataset:
https://data.gov.tw/dataset/102187

data url:
https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460/resource/662f822c-632b-4cb3-80f6-b5ac053d210b/download/11311.json

In [ ]:
import requests


url = "https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460/resource/662f822c-632b-4cb3-80f6-b5ac053d210b/download/11311.json"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
data

In [24]:
keys = data[0].keys()
keys

dict_keys(['seq', '車站編號', '車站中文名稱', '車站英文名稱', 'Latitude', 'Longitude'])

In [26]:
with open('station.csv', 'w', encoding='utf-8') as f:
    header = ','.join(keys)
    f.write(header + '\n')
    for row in data:
        values = [str(row.get(key, '')) for key in keys]
        values = ','.join(values)
        f.write(values + '\n')

In [27]:
from os import write
import csv
with open('station.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(data)